In [5]:
# pip install geokakao

In [2]:
import requests
import pandas as pd
import geopandas as gpd

In [3]:
df = pd.read_csv('./data/restaurant_list.csv')

In [4]:
df.head()

,food,store_name,addr,tel,re_rank1,re_rank2,re_rank3,re_rank4,re_rank5,re_visitor
0,간장게장,춘남식당,서울 마포구 성미산로 97 춘남식당,0507-1384-0693,음식이 맛있어요 35,재료가 신선해요 28,양이 많아요 24,친절해요 23,특별한 메뉴가 있어요 12,"꽃게탕 소자에 칼국수사리, 볶음밥을 추가해서 먹었어요! 연평도산 꽃게라 그런지 야들..."
1,간장게장,장안동 먹깨비 간장게장 해물아구찜,서울 동대문구 장한로 92 1층,0507-1393-2158,음식이 맛있어요 456,친절해요 225,재료가 신선해요 202,양이 많아요 164,매장이 넓어요 115,남친 부모님이 매운걸 못드셔서 안맵게로 부탁 존나 맵게해야 더 맛있을듯 이모님 사장...
2,간장게장,예담밥상,서울 광진구 동일로22길 13,0507-1443-0073,음식이 맛있어요 96,재료가 신선해요 52,친절해요 42,비싼 만큼 가치있어요 33,차분한 분위기에요 30,🦀간장게장과 갈비가 같이 나오는 특선을 추천 해주셔서 먹었습니다~ ^^ 확실히 찬들...
3,간장게장,명현만간장게장무한리필,서울 마포구 양화로 45 메세나폴리스 123호~129호,0507-1302-3837,음식이 맛있어요 677,재료가 신선해요 385,친절해요 370,양이 많아요 328,가성비가 좋아요 315,명현만의 간장게장 합정점! 평소 명현만 선수의 팬이었다면 더욱 좋을 것 같아요! 가...
4,간장게장,진미식당,서울 마포구 마포대로 186-6,02-3211-4468,음식이 맛있어요 364,재료가 신선해요 180,특별한 메뉴가 있어요 105,친절해요 73,특별한 날 가기 좋아요 41,맛있어요~예약이 힘들었는데 다행히 성공했어요ㅋ 들어가서 앉고 거의바로 음식이나왔어요...


In [15]:
# 카카오 API KEY
API_KEY = "98614acfa7370dcb5a0d9e815d6c9e5b"

In [16]:
def convert_address_to_coordinates(address):
    api_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {API_KEY}"}

    params = {"query": address}
    
    try:
        response = requests.get(api_url, headers=headers, params=params)
        response.raise_for_status()
        result = response.json()

        if "documents" in result and len(result["documents"]) > 0:
            coordinates = result["documents"][0]["y"], result["documents"][0]["x"]
            return coordinates
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [17]:
def add_coordinates_to_dataframe(df, address_column):
    latitudes = []
    longitudes = []

    for address in df[address_column]:
        coordinates = convert_address_to_coordinates(address)
        if coordinates:
            latitudes.append(coordinates[0])
            longitudes.append(coordinates[1])
        else:
            latitudes.append(None)
            longitudes.append(None)

    df["decimalLatitude"] = latitudes
    df["decimalLongitude"] = longitudes

In [19]:
# 주소를 좌표로 변환하여 데이터프레임에 추가
add_coordinates_to_dataframe(df, 'addr')
df.to_csv('korea_np.csv', index=False, encoding='utf-8')
df.head()

,food,store_name,addr,tel,re_rank1,re_rank2,re_rank3,re_rank4,re_rank5,re_visitor,decimalLatitude,decimalLongitude
0,간장게장,춘남식당,서울 마포구 성미산로 97 춘남식당,0507-1384-0693,음식이 맛있어요 35,재료가 신선해요 28,양이 많아요 24,친절해요 23,특별한 메뉴가 있어요 12,"꽃게탕 소자에 칼국수사리, 볶음밥을 추가해서 먹었어요! 연평도산 꽃게라 그런지 야들...",37.5615896264821,126.918127413756
1,간장게장,장안동 먹깨비 간장게장 해물아구찜,서울 동대문구 장한로 92 1층,0507-1393-2158,음식이 맛있어요 456,친절해요 225,재료가 신선해요 202,양이 많아요 164,매장이 넓어요 115,남친 부모님이 매운걸 못드셔서 안맵게로 부탁 존나 맵게해야 더 맛있을듯 이모님 사장...,37.5688108189618,127.069843827191
2,간장게장,예담밥상,서울 광진구 동일로22길 13,0507-1443-0073,음식이 맛있어요 96,재료가 신선해요 52,친절해요 42,비싼 만큼 가치있어요 33,차분한 분위기에요 30,🦀간장게장과 갈비가 같이 나오는 특선을 추천 해주셔서 먹었습니다~ ^^ 확실히 찬들...,37.5425201943402,127.065222308231
3,간장게장,명현만간장게장무한리필,서울 마포구 양화로 45 메세나폴리스 123호~129호,0507-1302-3837,음식이 맛있어요 677,재료가 신선해요 385,친절해요 370,양이 많아요 328,가성비가 좋아요 315,명현만의 간장게장 합정점! 평소 명현만 선수의 팬이었다면 더욱 좋을 것 같아요! 가...,37.5515008030203,126.91395024016
4,간장게장,진미식당,서울 마포구 마포대로 186-6,02-3211-4468,음식이 맛있어요 364,재료가 신선해요 180,특별한 메뉴가 있어요 105,친절해요 73,특별한 날 가기 좋아요 41,맛있어요~예약이 힘들었는데 다행히 성공했어요ㅋ 들어가서 앉고 거의바로 음식이나왔어요...,37.550607750987,126.955802324859


In [20]:
df.head(20)

,food,store_name,addr,tel,re_rank1,re_rank2,re_rank3,re_rank4,re_rank5,re_visitor,decimalLatitude,decimalLongitude
0,간장게장,춘남식당,서울 마포구 성미산로 97 춘남식당,0507-1384-0693,음식이 맛있어요 35,재료가 신선해요 28,양이 많아요 24,친절해요 23,특별한 메뉴가 있어요 12,"꽃게탕 소자에 칼국수사리, 볶음밥을 추가해서 먹었어요! 연평도산 꽃게라 그런지 야들...",37.5615896264821,126.918127413756
1,간장게장,장안동 먹깨비 간장게장 해물아구찜,서울 동대문구 장한로 92 1층,0507-1393-2158,음식이 맛있어요 456,친절해요 225,재료가 신선해요 202,양이 많아요 164,매장이 넓어요 115,남친 부모님이 매운걸 못드셔서 안맵게로 부탁 존나 맵게해야 더 맛있을듯 이모님 사장...,37.5688108189618,127.069843827191
2,간장게장,예담밥상,서울 광진구 동일로22길 13,0507-1443-0073,음식이 맛있어요 96,재료가 신선해요 52,친절해요 42,비싼 만큼 가치있어요 33,차분한 분위기에요 30,🦀간장게장과 갈비가 같이 나오는 특선을 추천 해주셔서 먹었습니다~ ^^ 확실히 찬들...,37.5425201943402,127.065222308231
3,간장게장,명현만간장게장무한리필,서울 마포구 양화로 45 메세나폴리스 123호~129호,0507-1302-3837,음식이 맛있어요 677,재료가 신선해요 385,친절해요 370,양이 많아요 328,가성비가 좋아요 315,명현만의 간장게장 합정점! 평소 명현만 선수의 팬이었다면 더욱 좋을 것 같아요! 가...,37.5515008030203,126.91395024016
4,간장게장,진미식당,서울 마포구 마포대로 186-6,02-3211-4468,음식이 맛있어요 364,재료가 신선해요 180,특별한 메뉴가 있어요 105,친절해요 73,특별한 날 가기 좋아요 41,맛있어요~예약이 힘들었는데 다행히 성공했어요ㅋ 들어가서 앉고 거의바로 음식이나왔어요...,37.550607750987,126.955802324859
5,간장게장,본가진미간장게장,서울 송파구 백제고분로 420 1층,0507-1342-5081,음식이 맛있어요 754,재료가 신선해요 431,친절해요 347,매장이 넓어요 230,양이 많아요 206,방이동 간장게장 최고 맛집이에요 !! 암꽃게 간장모듬장이랑 양념모듬장 하나씩 주문해...,37.5083270530231,127.111312208939
6,간장게장,보스 간장게장 서울식물원점,서울 강서구 마곡동로 161 서울식물원 4층,0507-1377-1201,음식이 맛있어요 97,매장이 넓어요 53,재료가 신선해요 51,뷰가 좋아요 36,친절해요 35,퇴근후 찾은 서울식물원 저녁메뉴 고민하다 서울식물원 맛집 보스간장게장을 갔다 알베기...,37.5722444079878,126.83284283132
7,간장게장,장안동 먹깨비 간장게장 해물아구찜,서울 동대문구 장한로 92 1층,0507-1393-2158,음식이 맛있어요 456,친절해요 225,재료가 신선해요 202,양이 많아요 164,매장이 넓어요 115,남친 부모님이 매운걸 못드셔서 안맵게로 부탁 존나 맵게해야 더 맛있을듯 이모님 사장...,37.5688108189618,127.069843827191
8,간장게장,순미네행복게장 동대문점,서울 중구 을지로43길 38 2층 순미네행복게장,0507-1400-2059,음식이 맛있어요 214,가성비가 좋아요 115,재료가 신선해요 110,친절해요 93,매장이 넓어요 90,"동대문에 이런 게장 맛집이 있었다니..ㅎ 숫게장정식 은 깔끔하고, 양념게장은 살짝 ...",37.5677113760105,127.007765983844
9,간장게장,레알 간장게장 무한리필 본점,서울 강북구 덕릉로41길 42 1층 레알 간장게장 무한리필 본점,0507-1367-0378,음식이 맛있어요 313,가성비가 좋아요 194,재료가 신선해요 191,양이 많아요 170,친절해요 162,수유 간장게장 맛집이에요!~ 암게를 무한리필로 양념과 간장 두 가지 다 먹을 수 있...,37.6368480826814,127.030126161232
